## Execution Scripts

These are the cells that generate the experiment results, kept separately from the rest of the code so as to not re-run experiments.

### Experiment A

In [ ]:
import os, json
import yaml

GRID_FILE = "grids.yaml"
BASE_OUT = "results/grid_2025_08_15"   # date-stamp it
OUTPUT_GRANULARITY = "event"           # Experiment A wants per-event for the money plot
SEEDS = list(range(5))                 # match your CLI default

# Load grid config from YAML
with open(GRID_FILE, "r") as f:
    grid_config = yaml.safe_load(f)


combos = generate_combinations(grid_config)

# select 60 combinations randomly 
cases = random.sample(combos, 10)

print(f"{len(combos)} grid cells")
print(f"sampled {len(cases)} cells to run")

# (Optional) dry-run preview
for i, p in enumerate(cases[:5]):
    print(f"{i+1:2d}. {create_grid_id(p)} → {p}")

# Make output dirs
os.makedirs(os.path.join(BASE_OUT, "sweep"), exist_ok=True)

# Run the sweep using ExperimentRunner
all_csv = []
for i, params in enumerate(cases):
    print(f"\n=== Running cell {i+1}/{len(combos)}: {create_grid_id(params)} ===")
    csvs = run_parameter_combination(
        params=params,
        seeds=SEEDS,
        base_out_dir=BASE_OUT,
        output_granularity=OUTPUT_GRANULARITY,
        parallel=1,           # bump if your runner supports safe parallelism in-notebook
    )
    all_csv.extend(csvs)

# Persist a manifest like the CLI does (handy for analysis notebooks)
manifest = { create_grid_id(p): p for p in combos }
with open(os.path.join(BASE_OUT, "sweep", "manifest.json"), "w") as f:
    json.dump(manifest, f, indent=2)

print("\nDone. CSVs:", len(all_csv))
